In [1]:
using Pkg 
Pkg.instantiate()
using qAlgebra

Precompiling project...
    962.5 ms  ✓ qAlgebra
  1 dependency successfully precompiled in 1 seconds. 7 already precompiled.


In [2]:
qspace = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", operators=["A(!i)", "B(U,H,i)"], h=QubitPM(), i=(3, QubitPauli()), b=Ladder())

StateSpace: [α, β(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ]
   - SubSpace ["h"]: PM Qubit (Fermionic):  pₚ, mₚ, zₚ, Iₚ (identity)
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  xₚ, yₚ, zₚ, Iₚ (identity)
   - SubSpace ["b"]: Ladder (Bosonic):  p†, p
   - Op: A
   - Op: B(H,U)


In [3]:
base_operators(qspace)
base_operators("alpha", qspace)
base_operators("i", qspace)
base_operators("h", qspace)
base_operators("j", qspace)
base_operators("I", qspace)
base_operators("A", qspace)
println("Done") 

MethodError: MethodError: no method matching qEQ(::qAlgebra.qExpressions.qProd)
The type `qEQ` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  qEQ(!Matched::StateSpace, !Matched::Vector{<:qComposite})
   @ qAlgebra ~/Documents/PhD/Research/Projects/qAlgebra/src/qExpressions.jl:80
  qEQ(!Matched::StateSpace, !Matched::qComposite)
   @ qAlgebra ~/Documents/PhD/Research/Projects/qAlgebra/src/qExpressions.jl:89
  qEQ(!Matched::StateSpace, !Matched::qAtom)
   @ qAlgebra ~/Documents/PhD/Research/Projects/qAlgebra/src/qExpressions.jl:92


In [ ]:
function term_equal_indexes(abstract::qAbstract, index1::Int, index2::Int, subspace::SubSpace)::Tuple{Bool, Vector{qAbstract}}
end

In [5]:
custom_sort_key

UndefVarError: UndefVarError: `custom_sort_key` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
term("alpha*A*xi*yi*b'*B*ph*mh")
println("Done")

Done


In [ ]:
function non_trivial(q::qAtom, index::

In [ ]:
function term_equal_indexes(prod::qProd, index1::Int, index2::Int, subspace::SubSpace)::Tuple{Bool, Vector{qProd}}
    changed_any = false
    term_variants = Vector{Vector{qAtom}}()
    for atom in prod.expr
        changed, variants = term_equal_indexes(atom, index1, index2, subspace)
        push!(term_variants, variants)
        changed_any |= changed  # Check if any term was changed
    end

    if !changed_any
        return false, [prod]
    end

    # Generate all combinations (cartesian product) of updated terms
    combinations = Iterators.product(term_variants...)

    simplified_products = qProd[]

    for combo in combinations
        new_expr = collect(combo)
        new_prod = qProd(prod.coeff_fun, new_expr)
        push!(simplified_products, simplify(new_prod))
    end

    return true, simplified_products
end


UndefVarError: UndefVarError: `custom_sort_key` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:
using ComplexRationals
a = ComplexRational(1,0,1)
copy(a)

1

In [ ]:
expr = 2 * alpha * im * xi + alpha * Dag(b) * xi * yi

In [ ]:
qsum = Sum("j",  alpha*yi*yj+Sum("k", beta*alpha^2*xi*xj*xk))

In [ ]:
flat_sum = flatten(qsum)

In [ ]:
# The sum still covers all combinations of indexes j,k
# We can transform it into a neq sum, in which the indexes j and k are distinct. the following function then expands into all possible cases
neq_sum = neq(qsum) # this also flattens the sum

In [ ]:
# A differential equation of expectation values can be constructed via
dzi_dt = d_dt(zi, alpha*expr+qsum)

In [ ]:
clipboard(string(dzi_dt))

In [ ]:
clipboard(latex_string(dzi_dt))